# Tutorial: PyTorch to CasADi NMPC via ONNX
## Part 3: Closed-Loop Simulation and Analysis

In Part 1, we trained a PyTorch ANN and exported it to ONNX. In Part 2, we imported this ONNX model into CasADi and formulated a Nonlinear Model Predictive Control (NMPC) problem using this ANN as the predictive model.

Now, in Part 3, we will:
1.  Implement the NMPC receding horizon control loop.
2.  Simulate the "true" Van der Pol oscillator (our plant) being controlled by the ANN-NMPC.
3.  Log and visualize the closed-loop performance: state trajectories, control inputs, and constraint satisfaction.
4.  Discuss the results and potential considerations when using data-driven models in MPC.

**Prerequisites:** Completion of Part 1 and Part 2.

### 3.1 Importing Libraries and Re-using Setup from Part 2

We'll bring in the necessary libraries and the CasADi `Opti()` object (`opti_vdp`) and the ONNX wrapper function (`f_ann_mpc_casadi`) that we created in Part 2. We also need the true plant model (`vdp_ode`) and its parameters.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import casadi as ca
from sklearn.preprocessing import MinMaxScaler # For scalers
import joblib # For loading scalers
import onnx # For ONNX model path definition
import torch # For loading PyTorch model to get weights if needed (for re-implementation method)
import torch.nn as nn # For model definition if re-implementing for verification

plt.rcParams.update({'font.size': 12, 'figure.figsize': (12, 8)}) 

# --- Parameters and Functions from Part 1 & 2 ---
# Van der Pol parameters
mu_vdp = 1.0
n_x = 2 # x1, x2
n_u = 1 # u
Ts_mpc_vdp = 0.1 # Control interval, should match ANN training data sampling if ANN predicts x_k+1

def vdp_ode(t, x_state, u_input, mu):
    x1, x2 = x_state
    dx1_dt = x2
    dx2_dt = mu * (1 - x1**2) * x2 - x1 + u_input
    return [dx1_dt, dx2_dt]

# Load scalers (essential!)
try:
    input_feature_scaler = joblib.load('input_feature_scaler.pkl')
    output_target_scaler = joblib.load('output_target_scaler.pkl')
    print("Loaded scalers from Part 1.")
except FileNotFoundError:
    print("ERROR: Scaler .pkl files not found. Please run Part 1 first to generate and save them!")
    # Create dummy scalers to allow rest of notebook to be defined, but it won't work correctly.
    input_feature_scaler = MinMaxScaler(feature_range=(-1, 1)); input_feature_scaler.fit(np.random.rand(10, n_x + n_u))
    output_target_scaler = MinMaxScaler(feature_range=(-1, 1)); output_target_scaler.fit(np.random.rand(10, n_x))

onnx_model_filepath = "vdp_ann_model.onnx"
ann_input_dim = n_x + n_u
ann_output_dim = n_x

# --- CasADi ONNX Function and Wrapper (from Part 2) ---
f_ann_mpc_casadi = None
try:
    onnx_casadi_func_scaled = ca.external('f_onnx_scaled_part3', onnx_model_filepath) # Use a new name if run in same session
    input_min_np = input_feature_scaler.min_
    input_scale_np = input_feature_scaler.scale_
    output_min_np = output_target_scaler.min_
    output_scale_np = output_target_scaler.scale_

    sx_k_unscaled = ca.SX.sym('sx_k_unscaled_w', n_x)
    su_k_unscaled = ca.SX.sym('su_k_unscaled_w', n_u)
    s_ann_input_unscaled = ca.vertcat(sx_k_unscaled, su_k_unscaled)
    s_ann_input_scaled = (s_ann_input_unscaled - input_min_np.reshape(-1,1)) * input_scale_np.reshape(-1,1)
    s_ann_output_scaled = onnx_casadi_func_scaled(s_ann_input_scaled)
    sx_k_plus_1_unscaled = s_ann_output_scaled / output_scale_np.reshape(-1,1) + output_min_np.reshape(-1,1)
    
    f_ann_mpc_casadi = ca.Function('f_ann_mpc_final', 
                                   [sx_k_unscaled, su_k_unscaled], 
                                   [sx_k_plus_1_unscaled],
                                   ['xk', 'uk'], ['xk_plus_1'])
    print("CasADi ONNX wrapper function ready.")
except Exception as e:
    print(f"Error creating CasADi ONNX wrapper: {e}. NMPC simulation will not run.")

# --- NMPC Formulation (from Part 2) ---
opti_vdp = None # Will be defined if f_ann_mpc_casadi is available
if f_ann_mpc_casadi:
    Np_vdp = 15
    Q_x1_vdp = 20.0; Q_x2_vdp = 2.0; R_u_vdp = 0.05; S_u_vdp = 0.1
    u_min_vdp = -2.0; u_max_vdp = 2.0; delta_u_max_vdp = 0.5
    x1_min_vdp = -2.5; x1_max_vdp = 2.5; x2_min_vdp = -3.0; x2_max_vdp = 3.0
    x1_sp_target_vdp = 0.0; x2_sp_target_vdp = 0.0

    opti_vdp = ca.Opti()
    X_sym_vdp = opti_vdp.variable(n_x, Np_vdp + 1)
    U_sym_vdp = opti_vdp.variable(n_u, Np_vdp)
    x0_vdp_param = opti_vdp.parameter(n_x)
    u_prev_vdp_param = opti_vdp.parameter(n_u)
    x1_sp_vdp_param = opti_vdp.parameter(Np_vdp)
    x2_sp_vdp_param = opti_vdp.parameter(Np_vdp)
    obj_vdp = 0
    for j in range(Np_vdp):
        obj_vdp += Q_x1_vdp * (X_sym_vdp[0, j+1] - x1_sp_vdp_param[j])**2
        obj_vdp += Q_x2_vdp * (X_sym_vdp[1, j+1] - x2_sp_vdp_param[j])**2
        obj_vdp += R_u_vdp * (U_sym_vdp[0, j])**2
        delta_u_vdp = U_sym_vdp[0, j] - (u_prev_vdp_param[0] if j==0 else U_sym_vdp[0, j-1])
        obj_vdp += S_u_vdp * delta_u_vdp**2
    opti_vdp.minimize(obj_vdp)
    opti_vdp.subject_to(X_sym_vdp[:,0] == x0_vdp_param)
    for j in range(Np_vdp):
        x_next_pred_vdp = f_ann_mpc_casadi(X_sym_vdp[:,j], U_sym_vdp[:,j])
        opti_vdp.subject_to(X_sym_vdp[:,j+1] == x_next_pred_vdp)
        opti_vdp.subject_to(opti_vdp.bounded(u_min_vdp, U_sym_vdp[0,j], u_max_vdp))
        delta_u_constr_vdp = U_sym_vdp[0,j] - (u_prev_vdp_param[0] if j==0 else U_sym_vdp[0,j-1])
        opti_vdp.subject_to(opti_vdp.bounded(-delta_u_max_vdp, delta_u_constr_vdp, delta_u_max_vdp))
        opti_vdp.subject_to(opti_vdp.bounded(x1_min_vdp, X_sym_vdp[0,j+1], x1_max_vdp))
        opti_vdp.subject_to(opti_vdp.bounded(x2_min_vdp, X_sym_vdp[1,j+1], x2_max_vdp))
    nlp_opts_vdp = {'ipopt.print_level': 0, 'print_time': 0, 'ipopt.max_iter': 150,
                    'ipopt.acceptable_tol': 1e-5, 'ipopt.acceptable_obj_change_tol': 1e-5}
    opti_vdp.solver('ipopt', nlp_opts_vdp)
    print("NMPC problem for VDP ANN-MPC ready.")

ERROR: Scaler .pkl files not found. Please run Part 1 first to generate and save them!
Error creating CasADi ONNX wrapper: .../casadi/core/casadi_os.cpp:166: Assertion "handle!=nullptr" failed:
DllLibrary::init_handle: Cannot load shared library 'vdp_ann_model.onnx': 
   (
    Searched directories: 1. casadipath from GlobalOptions
                          2. CASADIPATH env var
                          3. PATH env var (Windows)
                          4. LD_LIBRARY_PATH env var (Linux)
                          5. DYLD_LIBRARY_PATH env var (osx)
    A library may be 'not found' even if the file exists:
          * library is not compatible (different compiler/bitness)
          * the dependencies are not found
   )
  Tried '/home/tensor/Model_predictive_control/.venv/lib/python3.12/site-packages/casadi' :
    Error code: /home/tensor/Model_predictive_control/.venv/lib/python3.12/site-packages/casadi/vdp_ann_model.onnx: cannot open shared object file: No such file or directory
  Trie

### 3.2 Implementing the Receding Horizon Control Loop

In [2]:
if f_ann_mpc_casadi and opti_vdp:
    # Simulation Parameters
    sim_duration = 10.0 # seconds
    num_sim_steps = int(sim_duration / Ts_mpc_vdp)

    # Plant initial condition and previous input
    x_plant_current = np.array([1.5, 0.5]) # Start away from origin
    u_plant_prev = np.array([0.0])

    # Setpoint trajectory (stabilize to origin)
    x1_sp_horizon = np.full(Np_vdp, x1_sp_target_vdp)
    x2_sp_horizon = np.full(Np_vdp, x2_sp_target_vdp)

    # Data Logging
    t_history = np.zeros(num_sim_steps + 1)
    x_history_plant = np.zeros((n_x, num_sim_steps + 1))
    u_history_mpc = np.zeros((n_u, num_sim_steps))
    solver_time_history = np.zeros(num_sim_steps)

    x_history_plant[:, 0] = x_plant_current
    t_history[0] = 0

    # Initial guess for NMPC solver (can be important for convergence)
    # Warm starting: use previous solution shifted
    U_guess_mpc = np.zeros((n_u, Np_vdp)) 
    X_guess_mpc = np.tile(x_plant_current.reshape(n_x,1), (1, Np_vdp + 1))

    print(f"Starting ANN-NMPC simulation for {num_sim_steps} steps...")
    for k_step in range(num_sim_steps):
        current_time_sim = k_step * Ts_mpc_vdp
        print(f"Sim Step {k_step+1}/{num_sim_steps} (t={current_time_sim:.2f}s)", end='\r')

        # Set parameters for the Opti problem
        opti_vdp.set_value(x0_vdp_param, x_plant_current)
        opti_vdp.set_value(u_prev_vdp_param, u_plant_prev)
        opti_vdp.set_value(x1_sp_vdp_param, x1_sp_horizon)
        opti_vdp.set_value(x2_sp_vdp_param, x2_sp_horizon)

        # Set initial guess for solver (warm start)
        opti_vdp.set_initial(X_sym_vdp, X_guess_mpc)
        opti_vdp.set_initial(U_sym_vdp, U_guess_mpc)

        try:
            sol = opti_vdp.solve()
            U_optimal_sequence = sol.value(U_sym_vdp)
            X_predicted_sequence = sol.value(X_sym_vdp) # For updating guess
            u_applied_to_plant = U_optimal_sequence[:, 0]
            solver_time_history[k_step] = sol.stats()['t_wall_total']

            # Update guess for next iteration (shift)
            X_guess_mpc = np.hstack((X_predicted_sequence[:, 1:], X_predicted_sequence[:, -1].reshape(n_x,1)))
            U_guess_mpc = np.hstack((U_optimal_sequence[:, 1:], U_optimal_sequence[:, -1].reshape(n_u,1)))

        except RuntimeError as e:
            print(f"\nSolver failed at step {k_step+1}: {e}. Using previous control.")
            u_applied_to_plant = u_plant_prev.flatten()
            solver_time_history[k_step] = np.nan # Indicate failure
            # Reset guess to something safe if solver fails repeatedly
            U_guess_mpc = np.zeros((n_u, Np_vdp))
            X_guess_mpc = np.tile(x_plant_current.reshape(n_x,1), (1, Np_vdp + 1))
            
        u_history_mpc[:, k_step] = u_applied_to_plant

        # Simulate true plant for one step
        plant_sol_step = solve_ivp(vdp_ode, 
                                   [current_time_sim, current_time_sim + Ts_mpc_vdp], 
                                   x_plant_current, 
                                   args=(u_applied_to_plant[0], mu_vdp), 
                                   method='RK45', dense_output=False, 
                                   t_eval=[current_time_sim + Ts_mpc_vdp])
        x_plant_current = plant_sol_step.y[:,-1]

        x_history_plant[:, k_step+1] = x_plant_current
        t_history[k_step+1] = current_time_sim + Ts_mpc_vdp
        u_plant_prev = u_applied_to_plant.reshape(n_u, 1)
        
    print("\nANN-NMPC simulation finished.")
else:
    print("ANN-NMPC simulation skipped due to earlier errors in ONNX wrapper setup.")

ANN-NMPC simulation skipped due to earlier errors in ONNX wrapper setup.


### 3.3 Visualization and Performance Analysis

In [3]:
if f_ann_mpc_casadi and opti_vdp:
    fig, axs = plt.subplots(n_x + 1, 1, figsize=(10, 8), sharex=True)
    fig.suptitle('ANN-NMPC Control of Van der Pol Oscillator', fontsize=16)

    # Plot states
    state_labels_vdp = ['$x_1$ (position)', '$x_2$ (velocity)']
    sp_labels_vdp = [x1_sp_target_vdp, x2_sp_target_vdp]
    for i in range(n_x):
        axs[i].plot(t_history, x_history_plant[i, :], 'b-', label=f'{state_labels_vdp[i]} (Plant)')
        axs[i].axhline(sp_labels_vdp[i], color='r', linestyle=':', label=f'Setpoint $x_{i+1,sp}$')
        axs[i].set_ylabel(state_labels_vdp[i])
        axs[i].grid(True); axs[i].legend()

    # Plot control input
    axs[n_x].step(t_history[:-1], u_history_mpc[0, :], 'k-', where='post', label='Control Input $u$ ($T_c$)')
    axs[n_x].axhline(u_max_vdp, color='m', linestyle='--', label='$u_{max}$')
    axs_n_x].axhline(u_min_vdp, color='m', linestyle='--', label='$u_{min}$')
    axs[n_x].set_ylabel('Input $u$')
    axs[n_x].set_xlabel('Time (s)')
    axs[n_x].grid(True); axs[n_x].legend()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # Plot solver times
    plt.figure(figsize=(10,4))
    plt.plot(t_history[:-1], solver_time_history * 1000, 'o-')
    plt.title('NMPC Solver Time per Step')
    plt.xlabel('Time (s)'); plt.ylabel('Solver Time (ms)')
    plt.grid(True); plt.show()
    print(f"Average solver time: {np.nanmean(solver_time_history)*1000:.2f} ms")
    print(f"Max solver time: {np.nanmax(solver_time_history)*1000:.2f} ms (Sample time Ts={Ts_mpc_vdp*1000} ms)")

SyntaxError: unmatched ']' (2779125785.py, line 17)

### 3.4 Discussion and Troubleshooting

*   **Performance:** How well did the ANN-NMPC stabilize the Van der Pol oscillator or track setpoints? Did it respect constraints?
*   **Model Accuracy:** The performance heavily relies on how well the ANN (imported via ONNX) approximates the true system dynamics within the operating region encountered during control. If the ANN model has significant errors, the MPC's predictions will be off, leading to suboptimal or unstable control.
*   **Extrapolation:** If the MPC drives the system into regions where the ANN was not well-trained, the ANN's predictions (and thus the control actions) can become unreliable. This is a key risk with data-driven models.
*   **Computational Time:** Monitor the solver time. Is it consistently within the sampling interval $T_s$? The complexity of the ANN (number of layers/neurons) directly impacts the evaluation time within each NLP iteration.
*   **Solver Issues:** 
    *   Did the NLP solver (IPOPT) converge at every step? Failures can occur due to poor initial guesses, model inaccuracies leading to difficult optimization landscapes, or ill-conditioning.
    *   Warm-starting (using the previous solution as a guess) is crucial for improving convergence speed and reliability.
*   **ONNX Operator Support:** The success of `ca.external` with an ONNX file depends on CasADi's ability to interpret all ONNX operators in the graph symbolically for AD. Simpler ANNs (e.g., using standard layers like `Linear`, `ReLU`, `Tanh`) are more likely to be fully supported than exotic custom layers.
*   **Scaling Consistency:** Double-check that the scaling applied in the CasADi wrapper exactly matches the scaling used during the PyTorch ANN training. Any mismatch here will lead to incorrect model predictions.

**Exercises:**
1.  Try different NMPC tuning weights ($Q, R, S$) or prediction horizons ($N_p$).
2.  Change the initial state of the plant. Does the ANN-NMPC still perform well?
3.  If the ANN from Part 1 wasn't perfectly trained, or if you use a simpler/smaller ANN, how does it affect the closed-loop control performance?
4.  (Advanced) Try to make one of the constraints very tight to see if the solver reports infeasibility or if performance degrades significantly.

## Conclusion of the Tutorial

This three-part tutorial demonstrated a complete workflow for:
1.  Training a neural network model for system dynamics in PyTorch.
2.  Exporting this model to the ONNX format.
3.  Importing the ONNX model into CasADi as a symbolic function.
4.  Creating a wrapper to handle data scaling/unscaling for the ONNX model within CasADi.
5.  Using this CasADi-wrapped ONNX model as the predictive core of an NMPC controller.
6.  Simulating the closed-loop ANN-NMPC system.

This ONNX interchange pathway is powerful because it allows leveraging the strengths of specialized frameworks: PyTorch for flexible neural network training and CasADi for efficient, structured nonlinear programming and optimal control.

While this example used a relatively simple FNN and system, the same principles apply to more complex neural network architectures and control problems, with the main challenges often being the robust training of the data-driven model and ensuring its reliable integration and differentiation within the NMPC solver.